In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set the path to your ChromeDriver
service = Service("C:/chromedriver-win64/chromedriver.exe")

# Initialize the WebDriver
driver = webdriver.Chrome(service=service)

# URL for the 2025 NBA season team stats
url = 'https://www.basketball-reference.com/leagues/NBA_2025.html'

# Open the webpage
driver.get(url)

from selenium.common.exceptions import TimeoutException

# Wait for the "Team Per Game Stats" table to load with increased timeout
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, 'per_game-team'))
    )
except TimeoutException:
    print("Loading took too much time!")
    driver.quit()

# Find the "Team Per Game Stats" table
table = driver.find_element(By.ID, 'per_game-team')

# Extract the table headers
headers = [th.text.strip() for th in table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')]

# Extract all rows in the table
rows = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

# Extract data from each row
team_stats = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    if cols:  # Only process rows with data
        team_stats.append([ele.text.strip() for ele in cols])

# Create a DataFrame from the extracted data
df = pd.DataFrame(team_stats, columns=headers[1:])  # Skip first column if it's empty or unnecessary

# Clean up the DataFrame (remove any empty rows or columns)
df = df.dropna()

# Save the DataFrame to a CSV file
csv_file = "NBA_2025_Team_Stats.csv"
df.to_csv(csv_file, index=False)

# Close the browser after scraping
driver.quit()

# Notify the user and display the first few rows of the DataFrame
print(f"Team stats have been saved to {csv_file}")
print(df.head())


Team stats have been saved to NBA_2025_Team_Stats.csv
                  Team   G     MP    FG   FGA   FG%    3P   3PA   3P%    2P  \
0    Memphis Grizzlies  47  240.0  45.3  93.1  .486  14.1  37.9  .374  31.1   
1  Cleveland Cavaliers  46  240.0  44.5  89.8  .495  16.2  41.1  .395  28.3   
2       Denver Nuggets  46  242.2  45.3  90.0  .503  11.8  31.0  .381  33.5   
3      New York Knicks  47  241.1  43.8  88.5  .494  13.2  35.0  .378  30.5   
4       Boston Celtics  47  242.7  41.5  90.9  .457  17.7  48.7  .364  23.8   

   ...   FT%   ORB   DRB   TRB   AST  STL  BLK   TOV    PF    PTS  
0  ...  .774  13.1  34.5  47.6  29.6  9.2  6.1  16.9  21.3  123.3  
1  ...  .780  10.3  33.8  44.1  29.0  8.3  4.5  13.0  18.2  121.8  
2  ...  .766  11.2  34.5  45.7  31.2  8.5  4.8  14.4  17.4  120.6  
3  ...  .808  10.9  32.6  43.5  27.5  8.0  3.9  13.1  17.1  117.8  
4  ...  .794  11.1  33.9  45.0  25.4  7.5  5.7  11.8  16.3  117.2  

[5 rows x 24 columns]


In [6]:
df.info

<bound method DataFrame.info of                       Team   G     MP    FG   FGA   FG%    3P   3PA   3P%  \
0        Memphis Grizzlies  47  240.0  45.3  93.1  .486  14.1  37.9  .374   
1      Cleveland Cavaliers  46  240.0  44.5  89.8  .495  16.2  41.1  .395   
2           Denver Nuggets  46  242.2  45.3  90.0  .503  11.8  31.0  .381   
3          New York Knicks  47  241.1  43.8  88.5  .494  13.2  35.0  .378   
4           Boston Celtics  47  242.7  41.5  90.9  .457  17.7  48.7  .364   
5         Sacramento Kings  46  242.7  43.3  91.1  .476  12.5  35.9  .350   
6            Chicago Bulls  47  240.5  43.0  92.1  .466  16.0  43.0  .372   
7    Oklahoma City Thunder  45  240.0  43.4  91.7  .473  13.7  38.4  .358   
8            Atlanta Hawks  46  241.6  42.2  92.0  .459  12.8  37.1  .346   
9           Indiana Pacers  45  241.1  43.3  88.5  .490  12.7  34.1  .372   
10        Dallas Mavericks  47  240.5  42.0  87.9  .478  13.0  35.2  .370   
11         Milwaukee Bucks  44  240.6  42.0 